In [ ]:
#!/usr/bin/env python3
# mqtt_subscriber.py

import json
import time
import threading
import pytz
import paho.mqtt.client as mqtt
from datetime import datetime
from jetbot import Robot        # 기존 notebook에서 쓰던 로봇 제어 모듈
from SCSCtrl import TTLServo    # 서보 제어 모듈

# --- 전역 변수 설정 ---
korea_tz      = pytz.timezone("Asia/Seoul")
BROKER_ADDR   = "172.20.10.6"   # 라즈베리파이5 IP (필요시 변경)
BROKER_PORT   = 1883
COMMAND_TOPIC = "AGV/command"

# 로봇 초기화
robot = Robot()

# 자동 움직임 및 종료 제어 이벤트
auto_event = threading.Event()
exit_event = threading.Event()

def auto_motion_loop():
    """auto_event가 set 되어 있는 동안 1초마다 '자동 움직임' 출력"""
    while not exit_event.is_set():
        if auto_event.is_set():
            now = datetime.now(korea_tz).strftime("%H:%M:%S")
            print(f"[{now}] 자동 움직임")
            
            time.sleep(1)
            
            #### 여기다가 자동 움직임 로직 제어
            
            # 실제 로봇 제어 예시:
            # robot.forward(0.2)
            
            #### 여기다가 자동 움직임 로직 제어
            
            
            
        else:
            time.sleep(0.1)
            
            #### 여기다가 자동 움직임 멈춤 로직 제어
            # break
            #### 여기다가 자동 움직임 멈춤 로직 제어
            

# --- MQTT 콜백 함수 ---
def on_connect(client, userdata, flags, rc):
    print(f"[{datetime.now(korea_tz)}] MQTT 연결 {'성공' if rc==0 else '실패 코드='+str(rc)}")
    if rc == 0:
        client.subscribe(COMMAND_TOPIC, qos=1)
        print(f"[{datetime.now(korea_tz)}] 구독 시작 → {COMMAND_TOPIC}")

def on_publish(client, userdata, mid):
    print(f"[{datetime.now(korea_tz)}] publish 완료, mid={mid}")

def on_message(client, userdata, msg):
    # 종료 상태면 무시
    if exit_event.is_set():
        return

    try:
        message = json.loads(msg.payload.decode("utf-8"))
    except json.JSONDecodeError:
        print("Invalid JSON:", msg.payload)
        return

    cmd = message.get("cmd_string", "")
    print(f"[{datetime.now(korea_tz)}] 수신 명령 → {cmd}")

    # 수동 제어
    if cmd == "go":
        print("→ AGV 전진"); robot.forward(0.4)
    elif cmd in ("mid", "stop"):
        print("→ AGV 정지"); robot.stop()
    elif cmd == "left":
        print("→ AGV 좌회전"); robot.left(0.3)
    elif cmd == "right":
        print("→ AGV 우회전"); robot.right(0.3)
    elif cmd == "back":
        print("→ AGV 후진"); robot.backward(0.4)

    # 자동 움직임
    elif cmd == "auto_start":
        auto_event.set()
        print("→ 자동 움직임 START")
    elif cmd == "auto_stop":
        auto_event.clear()
        print("→ 자동 움직임 STOP")

    # 종료 명령
    elif cmd == "exit":
        print("→ exit 수신, 구독 해제 및 종료")
        exit_event.set()
        client.unsubscribe(COMMAND_TOPIC)
        client.loop_stop()
        client.disconnect()

# --- 메인 루틴 ---
def main():
    # 1) 자동 움직임 스레드 시작
    threading.Thread(target=auto_motion_loop, daemon=True).start()

    # 2) MQTT 클라이언트 설정
    client = mqtt.Client()
    client.on_connect  = on_connect
    client.on_message  = on_message
    client.on_publish  = on_publish

    try:
        client.connect(BROKER_ADDR, BROKER_PORT)
    except Exception as e:
        print(f"[{datetime.now(korea_tz)}] 초기 연결 실패: {e}")
        return

    client.loop_start()

    # 3) 키보드 인터럽트로 종료 처리
    try:
        while not exit_event.is_set():
            time.sleep(0.5)
    except KeyboardInterrupt:
        print("KeyboardInterrupt! 구독 해제 및 종료 중...")
        exit_event.set()
        client.unsubscribe(COMMAND_TOPIC)
        client.loop_stop()
        client.disconnect()
    finally:
        print("프로그램 종료")

if __name__ == "__main__":
    main()


[2025-05-25 01:05:48.153622+09:00] MQTT 연결 성공
[2025-05-25 01:05:48.155529+09:00] 구독 시작 → AGV/command
[2025-05-25 01:05:59.580790+09:00] 수신 명령 → auto_start
→ 자동 움직임 START
[01:05:59] 자동 움직임
[01:06:00] 자동 움직임
[2025-05-25 01:06:00.809634+09:00] 수신 명령 → auto_stop
→ 자동 움직임 STOP
[2025-05-25 01:06:01.626315+09:00] 수신 명령 → go
→ AGV 전진
[2025-05-25 01:06:03.164729+09:00] 수신 명령 → left
→ AGV 좌회전
[2025-05-25 01:06:05.008228+09:00] 수신 명령 → go
→ AGV 전진
[2025-05-25 01:06:06.544477+09:00] 수신 명령 → right
→ AGV 우회전
[2025-05-25 01:06:08.080158+09:00] 수신 명령 → go
→ AGV 전진
[2025-05-25 01:06:10.985044+09:00] 수신 명령 → back
→ AGV 후진
[2025-05-25 01:06:15.862493+09:00] 수신 명령 → left
→ AGV 좌회전
[2025-05-25 01:06:17.500889+09:00] 수신 명령 → right
→ AGV 우회전
[2025-05-25 01:06:19.958490+09:00] 수신 명령 → back
→ AGV 후진
[2025-05-25 01:06:21.680752+09:00] 수신 명령 → mid
→ AGV 정지
